### python .xml to .json

In [ ]:
!pip install xmltodict

In [ ]:
import json
import xmltodict

with open("/content/drive/MyDrive/Lotte_sample/10054_60_s_9_meta.xml",'r') as f:
    xmlString = f.read()

jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
 
print("\nJSON output(output.json):")
print(jsonString)


JSON output(output.json):
{
    "comp_cd": {
        "identifier": {
            "identifier": "10054_60_s_9.jpg",
            "copyright": "LOTTE Data Communication Company"
        },
        "div_cd": {
            "item_cd": "03021200001",
            "item_no": "10054",
            "div_l": "\uc8fc\ub958",
            "div_m": "\uae30\ud0c0\uc8fc\ub958",
            "div_s": "\uce75\ud14c\uc77c",
            "div_n": "\uce75\ud14c\uc77c",
            "comp_nm": "\ub86f\ub370\uc8fc\ub958",
            "img_prod_nm": "\uc21c\ud558\ub9ac\uc18c\ub2e4\ud1a1\ubc14\ub098\ub098355ML",
            "volume": "355ML",
            "barcd": "8801030996231",
            "width": "7",
            "length": "7",
            "height": "12",
            "nutrition_info": "{\"1\ud68c\uc81c\uacf5\ub7c9\":\"\",\"\ucd1d\ub0b4\uc6a9\ub7c9(g)\":\"\",\"\ucd1d\ub0b4\uc6a9\ub7c9(mL)\":\"\",\"\uc5d0\ub108\uc9c0(\u3389)\":\"\",\"\ub2e8\ubc31\uc9c8(g)\":\"\",\"\uc9c0\ubc29(g)\":\"\",\"\ud0c4\uc218\ud654\ubb3c

### Image Copy & Paste Example

In [ ]:
import numpy as np
import cv2

## paste image
to_paste = np.full((2988, 2988, 3), 255, dtype=np.uint8) # 계산대 색으로 조정
print(to_paste.shape)

## copy image
to_copy = cv2.imread('/content/drive/MyDrive/Lotte_sample/10094_0_m_1.jpg', cv2.IMREAD_COLOR) 
print(to_copy.shape)
# reshape
imageHeight, imageWidth = to_copy.shape[:2]

resizeHeight = int(0.25 * imageHeight) # x 0.25 resize !!
resizeWidth = int(0.25 * imageWidth)

to_copy_resize = cv2.resize(to_copy, (resizeHeight, resizeWidth), interpolation = cv2.INTER_CUBIC)

## copy & paste
to_paste[500:500+resizeHeight, 500:500+resizeWidth] = to_copy_resize 

cv2.imwrite("/content/drive/MyDrive/Lotte_sample_result/final_img.jpg", to_paste)

(2988, 2988, 3)
(2988, 2988, 3)


True

### Sample Dataset Unzip

In [ ]:
%cd /content/drive/MyDrive/LOTTE_sample

/content/drive/MyDrive/LOTTE_sample


In [ ]:
!unzip -qq "/content/drive/MyDrive/LOTTE_sample/LOTTE_sample.zip"

### Data Augmentation Code (using Jacard Overlap)

In [1]:
!pip install xmltodict # xml2json 에 필요한 라이브러리 설치

## code for Jacard Overlap (IOU)
def find_intersection(set_1, set_2):
    """
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)

def find_jaccard_overlap(set_1, set_2):
    """
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)

In [5]:
import numpy as np
import cv2
import json
import xmltodict
import glob
import random
import math
import torch
from collections import OrderedDict

annotation_list = []

# to make _meta.xml information list
file_path = "/content/drive/MyDrive/LOTTE_sample/*/*/*" # sample 데이터 전부 사용
file_pathes = glob.glob(file_path)
file_list = [file for file in file_pathes if file.endswith("_meta.xml")]

# resize ratio
resize_ratio = 0.3 # 원본 상품이미지 축소 비율

image_no = 1 # number of image

# LOTTE_sample 1 epoch -> about 170 combination images
epoch = 30

for i in range (epoch):

    random.shuffle(file_list) # 합성이미지당 다양한 상품으로 구성 -> random 하게 섞기
    idx = 0 # file list index

    #-------------------------------------------------------------------------------
    while idx < len(file_list):
        print("current file_list index: " + str(idx))

        n_objects = random.randrange(5,11) # number of objects -> 5~10개
      
        ## to_paste image
        to_paste = np.full((2988, 2988, 3), (173, 255, 47), dtype=np.uint8) # 배경이미지 색 조정

        cnt = 0 # 현재 합성한 상품이미지 개수
        pre_boxes = torch.zeros(1, 4, dtype = torch.long) # IOU 비교를 위한 기존 boxes 좌표

        annotation_dict = {}

        categories = []
        resolutions = []
        boxes = []

        while True:

            if idx >= len(file_list):
                print("epoch: {} Done !!".format(i+1))
                break
            
            if cnt >= n_objects:
                print("complete to make Combination Image No." + str(image_no))
                break

            line = file_list[idx]
            idx = idx + 1

            # .xml to .json
            with open(line,'r') as f:
                xmlString = f.read()
            f.close()
            obj_Info = json.loads(json.dumps(xmltodict.parse(xmlString), indent=4))

            if 'dict' not in str(type(obj_Info['comp_cd']['annotation']['object'])): # 복수 상품이미지 고려 X
                continue
            
            cnt = cnt + 1

            # to_copy image
            to_copy = cv2.imread(line[:-9] + ".jpg", cv2.IMREAD_COLOR) 

            imageHeight, imageWidth = to_copy.shape[:2]

            resizeHeight = int(resize_ratio * imageHeight)
            resizeWidth = int(resize_ratio * imageWidth)

            to_copy_resize = cv2.resize(to_copy, (resizeHeight, resizeWidth), interpolation = cv2.INTER_CUBIC)
          
            # to_copy resize
            obj = obj_Info['comp_cd']['annotation']['object']

            dict_val_box = obj['bndbox'].values()
            list_box = list(dict_val_box)
            org_box = list(map(int, list_box))
            box = [round(j*resize_ratio) for j in org_box] 

            # random offset + IOU -> occlusion 확인
            n_of_trial = 0
            while True:
                ## 랜덤 좌표
                ver = random.randint(0, 2988 - (box[3] - box[1]))
                hor = random.randint(0, 2988 - (box[2] - box[0])) 

                new_box = []
                x_min = hor
                y_min = ver
                x_max = hor + (box[2] - box[0])
                y_max = ver + (box[3] - box[1])
                new_box.append([x_min, y_min, x_max, y_max])

                # overlap 검사 실시
                new_box = torch.tensor(new_box)
                overlap = find_jaccard_overlap(pre_boxes, new_box).view([-1])
                overlap = overlap.tolist()

                check = 1
                for val in overlap: 
                    if val > 0.05: check = 0 
                if check == 1: break # occlusion 발생 X -> while 문 탈출
                
                n_of_trial = n_of_trial + 1
                if n_of_trial >= 10: break # 10번 이상 시도시 중단
            if n_of_trial >=10: continue
            
            # overlap 비교를 위한 pre_boxes -> new_boxes 추가
            pre_boxes = torch.cat([pre_boxes, new_box], dim=0)
            
            # image copy & paste
            to_paste[ver:ver+(box[3]-box[1]), hor:hor+(box[2]-box[0])] = \
                                                        to_copy_resize[box[1]:box[3], box[0]:box[2]]

            # to make annotation file
            category = int(obj_Info['comp_cd']['div_cd']['item_cd']) # category
            # image_path = line[:-9] + ".jpg" # path
            
            resolution_dict = obj_Info['comp_cd']['annotation']['size'].values() 
            resolution = list(map(int, list(resolution_dict))) # resolution

            categories.append(category)
            # pathes.append(image_path)
            resolutions.append(resolution)
            boxes.append([hor, ver,hor+(box[2]-box[0]), ver+(box[3]-box[1])])

        # save final image
        image_path = "/content/drive/MyDrive/LOTTE_sample_result/image_" + str(image_no) + ".jpg"
        cv2.imwrite(image_path, to_paste)
        image_no = image_no + 1

        annotation_dict['categories'] = categories
        annotation_dict['path'] = image_path
        annotation_dict['resolutions'] = resolutions
        annotation_dict['boxes'] = boxes

        annotation_list.append(annotation_dict)

    # save final annotation
    if (i+1) % 5 == 0 or (i+1) == epoch:
        with open("/content/drive/MyDrive/LOTTE_annotation_" + str(image_no-1) +  ".json", "w") as f: 
            json.dump(annotation_list, f, indent=2, ensure_ascii=False)
        f.close()
    #-------------------------------------------------------------------------------


   


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
current file_list index: 307
complete to make Combination Image No.2545
current file_list index: 324
complete to make Combination Image No.2546
current file_list index: 336
complete to make Combination Image No.2547
current file_list index: 350
complete to make Combination Image No.2548
current file_list index: 361
complete to make Combination Image No.2549
current file_list index: 378
complete to make Combination Image No.2550
current file_list index: 395
complete to make Combination Image No.2551
current file_list index: 408
complete to make Combination Image No.2552
current file_list index: 426
complete to make Combination Image No.2553
current file_list index: 437
complete to make Combination Image No.2554
current file_list index: 451
complete to make Combination Image No.2555
current file_list index: 469
complete to make Combination Image No.2556
current file_list index: 479
complete to make Combination Image No.2557
current file_list index: 490

### Draw GT Boxes from "LOTTE_annotation_No.json" file

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import json

def Draw_GT(original_image, path, GT_group):
    
    # Annotate -> object별 annotation
    annotated_image = original_image.convert('RGB')
    draw = ImageDraw.Draw(annotated_image)
    font = ImageFont.truetype("./calibril.ttf", 15)

    draw.text(xy=[10.0 ,10.0], text=str(path).upper(), fill='white',
                  font=font) ## 좌측 상단 해당 이미지 path
    
    ## Drawing GT Box !!
    # Suppress specific classes, if needed
    for i in range(len(GT_group)):

        # Boxes
        box_location = [GT_group[i][0], GT_group[i][1], GT_group[i][2], GT_group[i][3]]
        box_location = list(map(float, box_location))
        draw.rectangle(xy=box_location, outline='#176bfc')
        draw.rectangle(xy=[l + 1. for l in box_location], outline='#176bfc')  # a second rectangle at an offset of 1 pixel to increase line thickness
            
        # draw.rectangle(xy=[l + 2. for l in box_location], outline=label_color_map[
        #     det_labels[i]])  # a third rectangle at an offset of 1 pixel to increase line thickness
        # draw.rectangle(xy=[l + 3. for l in box_location], outline=label_color_map[
        #     det_labels[i]])  # a fourth rectangle at an offset of 1 pixel to increase line thickness

        # Text
        text_size = font.getsize('GT'.upper())
        text_location = [box_location[0] + 2., box_location[3]]
        textbox_location = [box_location[0], box_location[3] + text_size[1], box_location[0] + text_size[0] + 4.,
                            box_location[3]]
        draw.rectangle(xy=textbox_location, fill='#176bfc')
        draw.text(xy=text_location, text='GT'.upper(), fill='white',
                  font=font)
        
    del draw
  
    return annotated_image


if __name__ == '__main__':

    with open('/content/drive/MyDrive/LOTTE_annotation_169.json', 'r') as f:
        json_data = json.load(f)
    f.close()
    
    # f = open("/content/drive/MyDrive/RGBTDv2/test_split.txt", 'r')
    # lines = f.readlines()

    for idx, image_dict in enumerate(json_data):

        original_image = Image.open(image_dict['pathes'], mode='r')
        original_image = original_image.convert('RGB')
        Draw_GT(original_image, image_dict['pathes'], image_dict['boxes']).save('/content/drive/MyDrive/LOTTE_sample_GT/GT_image_' + str(idx+1) +'.jpg','JPEG')

        if idx % 20 == 0: # 얼마나 진행되었는지 확인용
          print(idx)
    print("Making GT boxes on Combination Images Complete!")


0
20
40
60
80
100
120
140
160
Making GT boxes on Combination Images Complete!


### Make Video from  GT annotated images

In [ ]:
import re
import cv2
import os
import numpy as np
import natsort ## 문자열로 된 숫자를 정렬하고 싶을 때!!

if __name__ == '__main__':
    path = '/content/drive/MyDrive/LOTTE_sample_GT' # "ims/2/a"
    paths = [os.path.join(path , i ) for i in os.listdir(path) if re.search(".jpg$", i)]
    # paths = natsort.natsorted(paths)

    #pathIn= './ims/2/'
    pathOut = '/content/drive/MyDrive/LOTTE_samples_video.mp4'
    fps = 1

    frame_array = []
    for idx , path in enumerate(paths) :

        img = cv2.imread(path)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()


In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!ls

 2021.summer_URP				  LOTTE_annotation_169.json
 2021.URP.Summer-main_KWH.zip			  LOTTE_sample
'2021.URP.Summer-main_KWH.zip (Unzipped Files)'   LOTTE_sample_GT
'Colab Notebooks'				  LOTTE_sample_result
 Crowd_Human					  MNIST
 data_parsing.py				  PASCAL_VOC
 Detection					  RGBTDv2
 FASHION					 'RGBTDv2.zip (Unzipped Files)'
 kaggle.json					  SSD
 KAIST_PD					  URP_DATA
 KAIST_PD_PATH					  VGG16
 LOTTE_annotation_100.json
